In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torchvision import datasets
import torchvision.transforms as transforms
from tensorboardX import SummaryWriter

In [3]:
data_transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Resize(32),
        transforms.Normalize((0.5),(1.0))
    ]
)

train_data = datasets.MNIST(root= './', train= True, download= True, transform= data_transform)
test_data = datasets.MNIST(root= './', train= False, download= True, transform= data_transform)

In [4]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_data, batch_size= 32, shuffle= True)
test_loader = DataLoader(test_data, batch_size= 32)

In [5]:
data, label = next(iter(train_loader))
print(data.shape)

torch.Size([32, 1, 32, 32])


In [9]:
class Lenet(nn.Module):
    def __init__(self):
        super(Lenet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels= 1, out_channels= 6, kernel_size= 5, stride= 1)
        self.conv2 = nn.Conv2d(in_channels= 6, out_channels= 16, kernel_size= 5, stride= 1)
        self.conv3 = nn.Conv2d(in_channels= 16, out_channels= 120, kernel_size= 5, stride= 1)
        self.fc1 = nn.Linear(in_features= 120, out_features= 84)
        self.fc2 = nn.Linear(in_features= 84, out_features= 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.tanh(x)
        x = F.max_pool2d(x, 2, 2)

        x = self.conv2(x)
        x = F.tanh(x)
        x = F.max_pool2d(x, 2, 2)

        x = self.conv3(x)
        x = F.tanh(x)
        x = x.view(-1, 120)

        x = self.fc1(x)
        x = F.tanh(x)

        x = self.fc2(x)

        return x
    

model = Lenet()
model

Lenet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=120, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=10, bias=True)
)

In [7]:
from torchsummary import summary

summary(model, input_size= (1, 32, 32))

Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            156
├─Conv2d: 1-2                            2,416
├─Conv2d: 1-3                            48,120
├─Linear: 1-4                            10,164
├─Linear: 1-5                            850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            156
├─Conv2d: 1-2                            2,416
├─Conv2d: 1-3                            48,120
├─Linear: 1-4                            10,164
├─Linear: 1-5                            850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0

In [10]:
lr = 1e-3
optim = Adam(model.parameters(), lr= lr)
epochs = 10
criterion = nn.CrossEntropyLoss()

device = 'cuda' if torch.cuda.is_available else 'cpu'
model.to(device)

writer = SummaryWriter()

step = 0

for epoch in range(epochs):
    for data, label in train_loader:
        data, label = data.to(device), label.to(device)
        optim.zero_grad()
        pred = model(data)
        loss = criterion(pred, label)
        loss.backward()
        optim.step()
        writer.add_scalar('loss/train', loss, step)
        step += 1

    print(f'{epoch+1} loss: {loss.item()}')

1 loss: 0.4039618968963623
2 loss: 0.0985371470451355
3 loss: 0.017387568950653076
4 loss: 0.007855194620788097
5 loss: 0.017101043835282326
6 loss: 0.0018982278415933251
7 loss: 0.0005978188710287213
8 loss: 0.04054940119385719
9 loss: 0.016384029760956764
10 loss: 0.004465547390282154
